<h1 align=center><font size="5"> SVM (Support Vector Machines)</font></h1>


Neste notebook, você usará SVM (Support Vector Machines) para construir e treinar um modelo usando registros de células humanas e classificar as células se as amostras são benignas ou malignas.

O SVM funciona mapeando dados para um espaço de recursos de alta dimensão para que os pontos de dados possam ser categorizados, mesmo quando os dados não são linearmente separáveis. Um separador entre as categorias é encontrado, então os dados são transformados de tal forma que o separador pode ser desenhado como um hiperplano. Em seguida, as características dos novos dados podem ser usadas para prever o grupo ao qual um novo registro deve pertencer.

In [ ]:
import pandas as pd
import pylab as pl
import numpy as np
import scipy.optimize as opt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
%matplotlib inline 
import matplotlib.pyplot as plt

<h2 id="load_dataset">Carregar os dados de Câncer</h2>
O exemplo é baseado em um conjunto de dados que está publicamente disponível no UCI Machine Learning Repository (Asuncion e Newman 2007)[http://mlearn.ics.uci.edu/MLRepository.html]. O conjunto de dados consiste em centenas de registros de amostras de células humanas, cada um contendo os valores de um conjunto de características de células. Os campos em cada registro são:

| Nome        | Descrição                             |
| ----------- | ---------------------------           |
| ID          | Identificação                         |
| Clump       | espessura do aglomerado               |
| UnifSize    | Uniformidade do tamanho da célula     |
| UnifShape   | Uniformidade do formato da célula     |
| MargAdh     | Adesão marginal                       |
| SingEpiSize | Tamanho de célula epitelial única     |
| BareNuc     | Núcleos puros                         |
| BlandChrom  | Cromatina branda                      |
| NormNucl    | Nucléolos normais                     |
| Mit         | Mitoses                               |
| Class       | Benigno ou Maligno                    |


Para fins de simplificar, estamos usando um conjunto de dados que possui um número relativamente pequeno de preditores em cada registro.


### Carregar arquivo CSV


In [ ]:
cell_df = pd.read_csv("datasets/cellsample_dataset/cell_samples.csv")
cell_df.head()

O campo ID contém os identificadores do paciente. As características das amostras de células de cada paciente estão contidas nos campos Clump até Mit. Os valores são classificados de 1 a 10, sendo 1 o mais próximo de benigno.

O campo Class contém o diagnóstico, conforme confirmado por procedimentos médicos separados, se as amostras são benignas (valor = 2) ou malignas (valor = 4).

Vejamos a distribuição das classes com base na espessura do aglomerado e na uniformidade do tamanho da célula:

In [ ]:
ax = cell_df[cell_df['Class'] == 4][0:50].plot(kind='scatter', x='Clump', y='UnifSize', color='DarkBlue', label='malignant');
cell_df[cell_df['Class'] == 2][0:50].plot(kind='scatter', x='Clump', y='UnifSize', color='Yellow', label='benign', ax=ax);
plt.show()

## Seleção e pré processamento dos dados


Vamos primeiramente olhar para os tipos das colunas:


In [ ]:
cell_df.dtypes

Parece que a coluna **BareNuc** inclui alguns valores que não são numéricos. Podemos adaptar essas linhas:


In [ ]:
cell_df = cell_df[pd.to_numeric(cell_df['BareNuc'], errors='coerce').notnull()]
cell_df['BareNuc'] = cell_df['BareNuc'].astype('int')
cell_df.dtypes

In [ ]:
feature_df = cell_df[['Clump', 'UnifSize', 'UnifShape', 'MargAdh', 'SingEpiSize', 'BareNuc', 'BlandChrom', 'NormNucl', 'Mit']]
X = np.asarray(feature_df)
X[0:5]

Queremos que o modelo preveja o valor de Classe (ou seja, benigno (= 2) ou maligno (= 4)). Como este campo pode ter um de apenas dois valores possíveis, precisamos alterar seu nível de medição para refletir isso.

In [ ]:
cell_df['Class'] = cell_df['Class'].astype('int')
y = np.asarray(cell_df['Class'])
y [0:5]

## Conjunto de treino/teste


Ok, dividimos nosso conjunto de dados em conjunto de treinamento e teste:

In [ ]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=4)
print ('Conjunto de treino:', X_train.shape,  y_train.shape)
print ('Conjunto de teste:', X_test.shape,  y_test.shape)

<h2 id="modeling">Modelagem</h2>


O algoritmo SVM oferece uma escolha de funções de kernel para realizar seu processamento. Basicamente, o mapeamento de dados em um espaço dimensional superior é chamado de kernelling. A função matemática usada para a transformação é conhecida como função kernel e pode ser de diferentes tipos, como:

`` `
1. Linear
2. Polinômio
3. Função de base radial (RBF)
4. Sigmóide
`` `

Cada uma dessas funções tem suas características, seus prós e contras e sua equação, mas como não há uma maneira fácil de saber qual função tem o melhor desempenho com qualquer conjunto de dados, geralmente escolhemos funções diferentes e comparamos os resultados. Vamos apenas usar o padrão, RBF (Função de base radial) para este laboratório.


In [ ]:
from sklearn import svm
clf = svm.SVC(kernel='rbf')
clf.fit(X_train, y_train) 

Depois de ser ajustado, o modelo pode então ser usado para prever novos valores:

In [ ]:
yhat = clf.predict(X_test)
yhat [0:5]

<h2 id="evaluation">Avaliação</h2>


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import itertools

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Matriz de confusão',
                          cmap=plt.cm.Blues):
    """
    Esta função imprime e plota a matriz de confusão.
    A normalização pode ser aplicada definindo `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Matriz de confusão normalizada")
    else:
        print('Matriz de confusão sem normalização')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('Rótulo verdadeiro')
    plt.xlabel('Rótulo previsto')

In [ ]:
# Cálculo da matriz de confusão
cnf_matrix = confusion_matrix(y_test, yhat, labels=[2,4])
np.set_printoptions(precision=2)

print (classification_report(y_test, yhat))

# Plot da matriz de confusão não normalizada
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=['Benign(2)','Malignant(4)'],normalize= False,  title='Matriz de confusão')

Pode-se também facilmente usar o **f1_score** da biblioteca sklearn:


In [ ]:
from sklearn.metrics import f1_score
f1_score(y_test, yhat, average='weighted') 

Vamos obter o índice de jaccard para precisão:


In [ ]:
from sklearn.metrics import jaccard_similarity_score
jaccard_similarity_score(y_test, yhat)

<h2 id="practice">Exercício</h2>
Você pode reconstruir o modelo, mas desta vez com um kernel <b>linear</b>? Você pode usar a opção <b>kernel = 'linear'</b>, ao definir o svm. Como a precisão muda com a nova função do kernel?


In [ ]:
# Escreva seu código abaixo. Não se esqueça de pressionar Shift + Enter para executar a célula


<details><summary>Clique aqui para ver a solução</summary>

```python
clf2 = svm.SVC(kernel='linear')
clf2.fit(X_train, y_train) 
yhat2 = clf2.predict(X_test)
print("F1-score médio: %.4f" % f1_score(y_test, yhat2, average='weighted'))
print("Índice Jaccard: %.4f" % jaccard_similarity_score(y_test, yhat2))
```

</details>
